<a href="https://colab.research.google.com/github/opentensor/bittensor-docs/blob/main/Bittensor_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Bittensor Training.


In [3]:
# First install bittensor onto our runtime.
! pip install bittensor==1.0.4

     |████████████████████████████████| 133kB 6.6MB/s 
     |████████████████████████████████| 194kB 11.5MB/s 
     |████████████████████████████████| 13.7MB 323kB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 204kB 35.7MB/s 
     |████████████████████████████████| 235kB 33.2MB/s 
     |████████████████████████████████| 450kB 44.9MB/s 
     |████████████████████████████████| 1.3MB 46.0MB/s 
     |████████████████████████████████| 1.5MB 23.5MB/s 
     |████████████████████████████████| 245kB 43.2MB/s 
     |████████████████████████████████| 2.5MB 26.9MB/s 
     |████████████████████████████████| 184kB 40.6MB/s 
     |████████████████████████████████| 266kB 47.2MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
     |████████████████████████████████| 2.6MB 35.7MB/s 
     |████████████████████████████████| 1.9MB 38.7MB/s 
 

In [4]:
# Bittensor uses torch heavily as it's payload encoding and machine learning toolkit.
# We also use asyncio and must nest our asyncio loop inside the outer-colab-loop.
import bittensor
import torch
import nest_asyncio 
nest_asyncio.apply()

In [5]:
# Instantiating a wallet:

# Querying the bittensor network is free, however, users who contribute to the network attain ownership through the distribution of Tao.
# Tao increases your bandwidth in the network as miner-neurons (machines serving intelligence models) are more incentivized to respond to queries. 
# Tao also increases your learning potential as miner-neurons apply gradients from nodes with network power.

# Your balance is held in a "wallet" which maintains your crypto-graphic keys, one a "coldkey" that holds tokens and another the "hotkey" that controls your miner.
# The following lines create a wallet's hot and coldkey, however, don't worry about saving these keys, they won't be subscribed on the network or hold any tokens.
wallet = bittensor.wallet.Wallet(
    path = "~/.bittensor/wallets/",
    name = "test_wallet",
    hotkey = "test_hotkey"
)
wallet.create_new_coldkey(use_password=False)
wallet.create_new_hotkey() 


IMPORTANT: Store this mnemonic in a secure (preferable offline place), as anyone who has possesion of this mnemonic can use it to regenerate the key and access your tokens. 

The mnemonic to the new key is:

trash unfold coyote behave slender hand conduct venture social luggage spell tunnel

You can use the mnemonic to recreate the key in case it gets lost. The command to use to regenerate the key using this mnemonic is:
bittensor-cli regen --mnemonic trash unfold coyote behave slender hand conduct venture social luggage spell tunnel

Writing key to /root/.bittensor/wallets//test_wallet/coldkey

IMPORTANT: Store this mnemonic in a secure (preferable offline place), as anyone who has possesion of this mnemonic can use it to regenerate the key and access your tokens. 

The mnemonic to the new key is:

sea mobile describe globe vital truly company cradle typical green notable wealth

You can use the mnemonic to recreate the key in case it gets lost. The command to use to regenerate the k

In [6]:
# Creating Bittensor components:

# The Bittensor api is built from plug-and-play components, for this tutorial we will be using three of them:
#  1. Subtensor: An interface to the blockchain: allows us to query state and send transactions.
#
#  2. Metagraph: An object which maintains chain-state information (who is online, their weights, stake etc) as torch objects.
#
#  3. Dendrite: An object which maintains RPC connections to other peers in the system and allows us to make forward and backward queries.


# Create our Kusangi blockchain connection.
subtensor = bittensor.subtensor.Subtensor(
    wallet = wallet,
    network = 'kusanagi'
)

# Create our Metagraph chain state object.
# The metagraph take the subtensor connection as a parameter.
metagraph = bittensor.metagraph.Metagraph(
    wallet = wallet,
    subtensor = subtensor
)

# Create our dendrite RPC client.
# The dendrite needs the wallet and the metagraph.
dendrite_config = bittensor.dendrite.Dendrite.default_config()
dendrite_config.receptor.do_backoff = False
dendrite = bittensor.dendrite.Dendrite(
    config = dendrite_config,
    wallet = wallet,
    metagraph = metagraph,
)


In [9]:

# Syncing the metagraph:

# Weight and neuron information changes continually as the blockchain progresses. The Metagraph sync
# command will query for new information and serve it to you as torch objects which you can use in your training 
# regimes.
metagraph.sync()
print (metagraph)


Syncing metagraph:
..............................................................................................................................................................


Metagraph:
uid: None, inflation_rate: 0.5 block: 642308 n_neurons: 158 
           116     95       113     142      91       22       11       71       140      9        107    152    ...  35       4        97       117    86       36       39       55       112     61       133     24     
S         116.010 1615.491 194.683  194.826 1930.030 2958.056 2632.262 3649.191  195.030 2842.419 26.155 29.531  ... 2942.902 2443.126 1959.307 37.884 2146.056 2519.722 2402.693 3994.467 222.697 2528.800 110.697 2040.495
R           6.025 3233.907 200.707 2457.095 1492.067 1204.810 2163.259 1619.003 1885.851 3276.797 26.161  0.411  ... 2403.541 1221.360 1642.136 62.429 2899.808 1870.012 3267.760 1510.762  44.496 1392.975   7.699 1911.786
I           0.000    0.006   0.000    0.005    0.003    0.002    0.004    0.003    0

In [10]:
# Creating inputs:

# The Bittensor network is designed to be multi-modality and thus query it through multiple datatypes.
# However, the network was initially seeded only with TEXT, a modality where inputs need to be tokenized sequences of 
# natural language for instance "the cat was big and bob was a builder".

# Bittensor comes with a pre-built GPT byte encoder. 
# All messages should be encoded with this tokenizer.
tokenizer = bittensor.__tokenizer__()

# Example: Tokenizing text for a network query.
sentence = 'the quick brown fox jumped over the lazy dog\'s ectoplasm'
tokenized_sentence = tokenizer( [sentence] )['input_ids']
print ('tokenize( [\"', sentence, '\"]) =', tokenized_sentence)

tokenize( [" the quick brown fox jumped over the lazy dog's ectoplasm "]) = [[1169, 2068, 7586, 21831, 11687, 625, 262, 16931, 3290, 338, 46080, 20106, 8597]]


In [12]:
# Querying Neurons:

# Each miner has a unique endpoint which we pulled from the chain during metagraph.sync()
# Below, we get the endpoint information for Adam: the first miner with uid=0
adam = metagraph.neurons[metagraph.state.index_for_uid[0]]
print("\"Adam\" or endpoint 0:", '\n\n', adam)

# To query a peer, we use the dendrite, our RPC tool. Below we send our previously tokenized text to this endpoint 
# and recieved our result.
print ('Make query ->')
response, codes = dendrite.forward_text( 
    neurons = [ adam ],
    x = [ torch.tensor(tokenized_sentence) ]
)

# NOTE: For consitency, all requests must follow the same shape constraints.
# TEXT: [batch_length, sequence_length] 
# IMAGE = [batch_length, sequence_length, n_channels, x_size, y_size ] 
# TENSOR = [batch_length, bittensor.__network_dim__]
# And responses are always of shape [batch_length, sequence_length, bittensor.__network_dim__]
print ('\n')
print ('Adam\'s response: \n', response[0], '\n')
print ('Response shape: \n', response[0].shape, '\n')
print ('Return code: \n', codes, '\n')





"Adam" or endpoint 0: 

 version: "1.0.4"
public_key: "0x7ada25ae51ad2a885223422434d1797bf279930e289e54242af2c21cb679952c"
address: "99.238.136.56"
port: 8091
ip_type: 4

Make query ->


Adam's response: 
 tensor([[[ 1.0398, -1.5879,  0.6261,  ..., -0.8697,  1.2138, -1.2623],
         [ 1.0683, -1.6476,  0.7924,  ..., -1.0210,  1.3494, -1.3323],
         [ 0.9459, -1.6552,  0.7668,  ..., -0.9936,  1.2320, -1.2875],
         ...,
         [ 0.9366, -1.5019,  0.8458,  ..., -1.1191,  1.2650, -1.3201],
         [ 0.9569, -1.5828,  0.8104,  ..., -1.0607,  1.2619, -1.3189],
         [ 0.9157, -1.5109,  0.7737,  ..., -1.0389,  1.2615, -1.3511]]],
       grad_fn=<_ReceptorCallBackward>) 

Response shape: 
 torch.Size([1, 13, 512]) 

Return code: 
 tensor([0]) 



In [13]:
# Understanding queries:

# We just queried a single peer, Adam, and got a response. What happened?

# 1. Our tensor was serialized using a built in serializer class, converting the tensor into bytes.
#
# e.g. serializer = bittensor.serialization.get_serializer(bittensor.proto.Serializer.MSGPACK)
#.     serialized_tensor = serializer.serialize_from_torch( torch.tensor(tokenized_sentence), bittensor.proto.Modality.TEXT )


# 2. Our byte-encoded tensor was packaged into an RPC request and sent over the wire to our endpoint, in this case Adam's endpoint: 99.238.136.56:8091
#
# e.g. adam_receptor = list(dendrite.receptors)[0]
#.     adam_receptor.forward( torch.tensor(tokenized_sentence), bittensor.proto.Modality.TEXT)


# 3. Adam deserialized the request and used it as input to his transformer model. 
#    NOTE: Adam is running a custom GPT2 model trained on the genesis dataset for language modelling.
#
# e.g. deserializer = bittensor.serialization.get_serializer(bittensor.proto.Serializer.MSGPACK)
#      deserialized_tensor = serializer.deserialize_to_torch( serialized_tensor )


# 4. The output of Adam's transformer model is a sequence of representations, each
# representation of length bittensor.__network_dim__, one for each token in the sentence. 
# These are the standard hidden units of a transformer model and encode the meaning (according to Adam)
# of each token in it's position.
#
# e.g. response_tensor = AdamModel.forward_text( deserialized_tensor )
#      assert output_tensor.shape = [1, 13, bittensor.__network_dim__]


# 5. Adam's response tensor is serialized and returned to the sender. 

In [ ]:
# Putting it together:
# Below, we will train a custom model for Poem Sentiment Classification by querying the network.


# Load our dataset.
import datasets
dataset = datasets.load_dataset('poem_sentiment')
print ('\n\nExample sentence: \"', dataset['train']['verse_text'][3], '\"\n\nlabel: ', dataset['train']['label'][3])

Using custom data configuration default
Reusing dataset poem_sentiment (/root/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/f4990808f049126bcea572bba70613313212cd45f3b12a3e5586135e2de42f56)




Example sentence: " when i peruse the conquered fame of heroes, and the victories of mighty generals, i do not envy the generals, "

label:  3


In [ ]:
# Building the model.
import torch.nn as nn
import torch.nn.functional as F

class Pooler(nn.Module):
    def __init__(self):
        super(Pooler, self).__init__()
        self.dense = nn.Linear(bittensor.__network_dim__, bittensor.__network_dim__)
        self.activation = nn.Tanh()

    def forward(self, x: torch.FloatTensor):
        # Take last sequence encoding as the sentence's representation.
        first_representation = x[:, -1]
        pooled_output = self.dense(first_representation)
        pooled_output = self.activation(pooled_output)
        return pooled_output

class PoemSentimentClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        # For projecting sequences of representations into a single represenation.
        self.pooler = Pooler()

        # A Feedforward dense layer.
        self.hidden = nn.Linear(bittensor.__network_dim__, bittensor.__network_dim__)

        # For projecting our learned feature space onto the target dimension.
        self.target = nn.Linear(bittensor.__network_dim__, 4)

    def forward(self, x: torch.LongTensor):
        # Our model's forward call.

        # First, query every peer on kusanagi. (Slow for this tutorial)
        network_query = [ x for _ in metagraph.neurons]
        responses, _ = dendrite.forward_text( metagraph.neurons, network_query )

        # Average and pool responses.
        averaged_responses = torch.mean(torch.stack(responses, dim=2), dim=2)
        pooled_responses = self.pooler( averaged_responses )

        # Apply our dense layer and project it onto our target layer.
        hidden_layer = self.hidden( pooled_responses )
        logit_layer = self.target( hidden_layer )
        outputs = F.softmax( logit_layer, dim=1 )

        # Return our softmax-predictions.
        return outputs


In [ ]:
# Simple training architecture.
from typing import Tuple
import random

# Training params.
n_steps = 1000
batch_size = 100
learning_rate = 0.01
momentum = 0.99

# Model and optimizer.
tokenizer = bittensor.__tokenizer__()
model = PoemSentimentClassifier()
optimizer = torch.optim.SGD( model.parameters(), lr = learning_rate, momentum = momentum)
loss_function = torch.nn.CrossEntropyLoss(ignore_index=-1)

# Batch iterator: Produces random tokenized batches from the poem dataset.
def next_batch(batch_size: int, dataset, tokenizer) -> Tuple[torch.LongTensor, torch.LongTensor]:
  inputs = []
  targets = []
  for i in range(batch_size):
    random_index = random.randint(0, len(dataset)-1)
    inputs.append( dataset[random_index]['verse_text'] )
    targets.append( dataset[random_index]['label'] )
  inputs = tokenizer(inputs, return_tensors='pt', padding=True, truncation=True)['input_ids']
  targets = torch.tensor( targets, dtype=torch.int64 )
  return inputs, targets
  
# Training loop:
for batch_index in range(n_steps):
  inputs, targets = next_batch(batch_size, dataset['train'], tokenizer)
  logits = model( inputs )
  loss = loss_function( logits.view(-1, 4), targets )
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print ('step: ', batch_index, ' loss: ', loss.item())

            



step:  0  loss:  1.3818920850753784
step:  1  loss:  1.3808767795562744
step:  2  loss:  1.3803505897521973
step:  3  loss:  1.3786779642105103
step:  4  loss:  1.3771100044250488
step:  5  loss:  1.377078890800476
step:  6  loss:  1.3786886930465698
step:  7  loss:  1.375696063041687
step:  8  loss:  1.3745461702346802
step:  9  loss:  1.3682986497879028
step:  10  loss:  1.370833158493042
step:  11  loss:  1.3669792413711548
step:  12  loss:  1.3671680688858032
step:  13  loss:  1.3562805652618408
step:  14  loss:  1.3558220863342285
step:  15  loss:  1.3443596363067627
step:  16  loss:  1.3502075672149658
step:  17  loss:  1.3456476926803589
step:  18  loss:  1.3401801586151123
step:  19  loss:  1.3371249437332153
step:  20  loss:  1.3173526525497437
step:  21  loss:  1.3391228914260864
step:  22  loss:  1.3213753700256348
step:  23  loss:  1.3006536960601807
step:  24  loss:  1.2978423833847046
step:  25  loss:  1.2918426990509033
step:  26  loss:  1.2716795206069946
step:  27  los